# We want to convert text -> numerical values

The Goals are:
* Create Vocabulary mapping each word to an index
* We need to setup PyTorch dataset to load the data
* Setup padding of every padding (all example should be of the same sequence length)

In [1]:
import re
import pathlib
import numpy as np
from MSCOCO import COCO

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [6]:
root = '../../Datasets/MS COCO'
imgs_path = 'train2017'
captions_path = 'annotations_trainval2017/annotations/captions_train2017.json'

interface_params = {
    'root': root,
    'imgs_path': imgs_path,
    'captions_path': captions_path
}

In [7]:
voc = []
coco = COCO(**interface_params)

In [9]:
imgs_caps_dict = coco.imgs_caps_dict

In [ ]:
captions = []
for captions in imgs_caps_dict.values():
    for cap in captions:
        captions.append(cap)

In [ ]:
OOV = '<UNK>'
start_tokens = '<BOS>'
end_tokens = '<E>'

In [10]:
imgs_caps_dict

{'000000000009.jpg': ['Closeup of bins of food that include broccoli and bread.',
  'A meal is presented in brightly colored plastic trays.',
  'there are containers filled with different kinds of foods',
  'Colorful dishes holding meat, vegetables, fruit, and bread.',
  'A bunch of trays that have different food.'],
 '000000000025.jpg': ['A giraffe eating food from the top of the tree.',
  'A giraffe standing up nearby a tree ',
  'A giraffe mother with its baby in the forest.',
  'Two giraffes standing in a tree filled area.',
  'A giraffe standing next to a forest filled with trees.'],
 '000000000030.jpg': ['A flower vase is sitting on a porch stand.',
  'White vase with different colored flowers sitting inside of it. ',
  'a white vase with many flowers on a stage',
  'A white vase filled with different colored flowers.',
  'A vase with red and white flowers outside on a sunny day.'],
 '000000000034.jpg': ['A zebra grazing on lush green grass in a field.',
  'Zebra reaching its hea